In [4]:
with open("15.txt") as i:
    q_input = i.read().splitlines()

In [3]:
from dataclasses import dataclass

@dataclass
class Coord:
    x: int
    y: int
    def __add__(self, other):
        return Coord(self.x + other.x, self.y + other.y)
    def __sub__(self, other):
        return Coord(self.x - other.x, self.y - other.y)
    def __hash__(self):
        return hash((self.x, self.y))

GRID = {}
SCANNED = "#"
BEACON = "B"
SENSOR = "S"
P1_ROW = 10
#P1_ROW = 2_000_000

def fill_scan(sensor, dis, p1_row_dis):
    scan_widths = {row: dis - abs(row) for row in range(-dis,dis+1)}
    half_width = scan_widths[p1_row_dis]
    scan_row = sensor.y + p1_row_dis
    row_scan_coords=[Coord(x,scan_row) for x in range(sensor.x-half_width,sensor.x+half_width+1)]
    #print(row_scan_coords)
    for coord in row_scan_coords:
        grid_pos = GRID.get(coord,None)
        if grid_pos:
            continue
        GRID[coord]=SCANNED

def is_needed_sensor(sensor,dis):
    for row in range(-dis,dis+1):
        if sensor.y + row == P1_ROW:
            return True, row
    return False, None

def manhatten_distance(coord1, coord2):
    vector = coord2 - coord1
    return abs(vector.x)+abs(vector.y)

In [144]:
GRID={}
sensor_info={}
for line in q_input:
    line = line[12:]
    sensor_string, beacon_string = line.split(": closest beacon is at x=")
    sx, sy = sensor_string.split(", y=")
    bx, by = beacon_string.split(", y=")
    sensor_pos = Coord(int(sx),int(sy))
    beacon_pos = Coord(int(bx),int(by))
    GRID[sensor_pos]=SENSOR
    GRID[beacon_pos]=BEACON
    sensor_info[sensor_pos]=manhatten_distance(sensor_pos, beacon_pos)

print(len(sensor_info))

40


In [145]:
needed_sensors = {}
for sensor,distance in sensor_info.items():
    sensor_is_needed, p1_row_dis = is_needed_sensor(sensor,distance)
    if sensor_is_needed:
        needed_sensors[sensor]=(distance,p1_row_dis)

In [146]:
#Takes 20 sec!
for sensor,(b_dis,p1_row_dis) in needed_sensors.items():
    print("Filling scan from: ", sensor)
    fill_scan(sensor, b_dis, p1_row_dis)

Filling scan from:  Coord(x=1112863, y=496787)
Filling scan from:  Coord(x=760990, y=1455625)
Filling scan from:  Coord(x=2888433, y=2337157)
Filling scan from:  Coord(x=3423261, y=2191958)
Filling scan from:  Coord(x=139591, y=1186912)
Filling scan from:  Coord(x=1849154, y=1377259)
Filling scan from:  Coord(x=3811778, y=1370280)
Filling scan from:  Coord(x=3963152, y=2368927)
Filling scan from:  Coord(x=952643, y=2385401)
Filling scan from:  Coord(x=2306388, y=1932261)
Filling scan from:  Coord(x=1549467, y=3109269)
Filling scan from:  Coord(x=1189516, y=2153239)
Filling scan from:  Coord(x=468190, y=1889204)
Filling scan from:  Coord(x=270403, y=2762568)


In [148]:
p1_row_count = 0
for coord, val in GRID.items():
    if coord.y == P1_ROW:
        if val == SCANNED or val == SENSOR:
            p1_row_count +=1
print("Part 1:", p1_row_count)

Part 1: 5073496


# MY PART ONE METHOD WAS TERRIBLE

Basically unusable for part 2

# Part Two

In [16]:
GRID={}
BOUND = 4000000

sensor_info={}
for line in q_input:
    line = line[12:]
    sensor_string, beacon_string = line.split(": closest beacon is at x=")
    sx, sy = sensor_string.split(", y=")
    bx, by = beacon_string.split(", y=")
    sensor_pos = Coord(int(sx),int(sy))
    beacon_pos = Coord(int(bx),int(by))
    GRID[sensor_pos]=SENSOR
    GRID[beacon_pos]=BEACON
    sensor_info[sensor_pos]=manhatten_distance(sensor_pos, beacon_pos)

print(len(sensor_info))

40


In [17]:
class ScanRange():
    scan_ranges: list

    def __init__(self):
        self.scan_ranges = []

    def append(self, new_range):
        self.scan_ranges.sort()
        # print(self.scan_ranges)
        # print(new_range)
        new_start, new_end = new_range
        new_scan_ranges = []
        for start, end in self.scan_ranges:

            # Contains current range, ignore it
            if new_start <= start and new_end >= end:
                continue

            # Overlaps (right)
            elif new_start <= end+1 and new_end > end:
                new_start = start

            # Overlaps (left)
            elif new_end >= start-1 and new_start < start:
                new_end = end

            # New range is contained, expand out
            elif new_start >= start and new_end <= end:
                new_start = start
                new_end = end

            # No overlap, add range
            else:
                new_scan_ranges.append((start,end))
                
        new_scan_ranges.append((new_start,new_end))
        self.scan_ranges=new_scan_ranges
        self.scan_ranges.sort()
        
scan_ranges_list = []

for y in range(BOUND+1):
    scan_range = ScanRange()
    for sensor, man_dis in sensor_info.items():
        row_distance = y-sensor.y
        half_width = man_dis - abs(row_distance)
        if half_width >= 0:
            start = sensor.x - half_width
            end = sensor.x + half_width
            scanned_range = (start,end)
            scan_range.append(scanned_range)
    scan_ranges_list.append(scan_range)


Part 2: 13081194638237


In [20]:
distress_beacon = None
for i, scan in enumerate(scan_ranges_list):
    if len(scan.scan_ranges)==1:
        start,end=scan.scan_ranges[0]
        if start <= 0 and end >= BOUND:
            continue
        elif start > 0:
            distress_beacon = (0,i)
        elif end <= BOUND:
            distress_beacon = (BOUND,i)
    else:
        x = scan.scan_ranges[0][1] + 1
        distress_beacon = (x,i)

print("Part 2:", distress_beacon[0]*4000000 + distress_beacon[1])

Part 2: 13081194638237
